## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import pandas as pd
import tensorflow as tf
from keras.callbacks import Callback
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN","NAME"], 1)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_counts[application_counts < 1000].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C5200        1
C1580        1
C1248        1
C3700        1
C4200        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_count_great_one = class_counts[class_counts > 1]
class_count_great_one

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C1720        6
C4100        6
C2400        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C1234        2
C3200        2
C1246        2
C1256        2
C1267        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts < 1750].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df)
application_df_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df_dummies.drop(['IS_SUCCESSFUL'], axis = 'columns').values
y = application_df_dummies['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input_layer = len(X_train[0])
hidden_node_layers1 = 100
hidden_node_layers2 = 80
hidden_node_layers3 = 60
hidden_node_layers4 = 40
hidden_node_layers5 = 1

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_node_layers1, input_dim = num_input_layer, activation = 'linear'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_node_layers2, activation = 'linear'))

# Three
nn.add(tf.keras.layers.Dense(units = hidden_node_layers3, activation = 'relu'))

# Four
nn.add(tf.keras.layers.Dense(units = hidden_node_layers4, activation = 'sigmoid'))

# Output layer
nn.add(tf.keras.layers.Dense(units = hidden_node_layers5, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4100      
                                                                 
 dense_1 (Dense)             (None, 80)                8080      
                                                                 
 dense_2 (Dense)             (None, 60)                4860      
                                                                 
 dense_3 (Dense)             (None, 40)                2440      
                                                                 
 dense_4 (Dense)             (None, 1)                 41        
                                                                 
Total params: 19,521
Trainable params: 19,521
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Checkpoint file creation
os.makedirs('checkpoints/', exist_ok = True)
checkpoint_dir = r'checkpoints/Checkpoints.{epoch:02d}.hdf5'

In [14]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

checkpoint = ModelCheckpoint(filepath = checkpoint_dir, monitor = 'accuracy', verbose = 1, save_weights_only = True,
                            mode = 'auto', period = 1)

In [15]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 250, callbacks = [checkpoint])

Epoch 1/250
746/804 [==========================>...] - ETA: 0s - loss: 0.5729 - accuracy: 0.7156
Epoch 1: saving model to checkpoints\Checkpoints.01.hdf5
804/804 [==============================] - 1s 824us/step - loss: 0.5708 - accuracy: 0.7175
Epoch 2/250
800/804 [============================>.] - ETA: 0s - loss: 0.5613 - accuracy: 0.7279
Epoch 2: saving model to checkpoints\Checkpoints.02.hdf5
804/804 [==============================] - 1s 766us/step - loss: 0.5613 - accuracy: 0.7279
Epoch 3/250
782/804 [============================>.] - ETA: 0s - loss: 0.5570 - accuracy: 0.7265
Epoch 3: saving model to checkpoints\Checkpoints.03.hdf5
804/804 [==============================] - 1s 785us/step - loss: 0.5576 - accuracy: 0.7261
Epoch 4/250
803/804 [============================>.] - ETA: 0s - loss: 0.5565 - accuracy: 0.7277
Epoch 4: saving model to checkpoints\Checkpoints.04.hdf5
804/804 [==============================] - 1s 765us/step - loss: 0.5564 - accuracy: 0.7278
Epoch 5/250
773/804 

782/804 [============================>.] - ETA: 0s - loss: 0.5547 - accuracy: 0.7291
Epoch 34: saving model to checkpoints\Checkpoints.34.hdf5
804/804 [==============================] - 1s 783us/step - loss: 0.5547 - accuracy: 0.7290
Epoch 35/250
773/804 [===========================>..] - ETA: 0s - loss: 0.5530 - accuracy: 0.7287
Epoch 35: saving model to checkpoints\Checkpoints.35.hdf5
804/804 [==============================] - 1s 793us/step - loss: 0.5534 - accuracy: 0.7283
Epoch 36/250
784/804 [============================>.] - ETA: 0s - loss: 0.5542 - accuracy: 0.7315
Epoch 36: saving model to checkpoints\Checkpoints.36.hdf5
804/804 [==============================] - 1s 784us/step - loss: 0.5550 - accuracy: 0.7310
Epoch 37/250
741/804 [==========================>...] - ETA: 0s - loss: 0.5541 - accuracy: 0.7302
Epoch 37: saving model to checkpoints\Checkpoints.37.hdf5
804/804 [==============================] - 1s 760us/step - loss: 0.5550 - accuracy: 0.7294
Epoch 38/250
744/804 [===

804/804 [==============================] - ETA: 0s - loss: 0.5582 - accuracy: 0.7290
Epoch 67: saving model to checkpoints\Checkpoints.67.hdf5
804/804 [==============================] - 1s 761us/step - loss: 0.5582 - accuracy: 0.7290
Epoch 68/250
804/804 [==============================] - ETA: 0s - loss: 0.5578 - accuracy: 0.7281
Epoch 68: saving model to checkpoints\Checkpoints.68.hdf5
804/804 [==============================] - 1s 764us/step - loss: 0.5578 - accuracy: 0.7281
Epoch 69/250
800/804 [============================>.] - ETA: 0s - loss: 0.5568 - accuracy: 0.7275
Epoch 69: saving model to checkpoints\Checkpoints.69.hdf5
804/804 [==============================] - 1s 771us/step - loss: 0.5569 - accuracy: 0.7276
Epoch 70/250
775/804 [===========================>..] - ETA: 0s - loss: 0.5581 - accuracy: 0.7293
Epoch 70: saving model to checkpoints\Checkpoints.70.hdf5
804/804 [==============================] - 1s 788us/step - loss: 0.5573 - accuracy: 0.7296
Epoch 71/250
804/804 [===

786/804 [============================>.] - ETA: 0s - loss: 0.5602 - accuracy: 0.7281
Epoch 100: saving model to checkpoints\Checkpoints.100.hdf5
804/804 [==============================] - 1s 848us/step - loss: 0.5597 - accuracy: 0.7284
Epoch 101/250
768/804 [===========================>..] - ETA: 0s - loss: 0.5593 - accuracy: 0.7287
Epoch 101: saving model to checkpoints\Checkpoints.101.hdf5
804/804 [==============================] - 1s 862us/step - loss: 0.5592 - accuracy: 0.7283
Epoch 102/250
767/804 [===========================>..] - ETA: 0s - loss: 0.5583 - accuracy: 0.7291
Epoch 102: saving model to checkpoints\Checkpoints.102.hdf5
804/804 [==============================] - 1s 803us/step - loss: 0.5579 - accuracy: 0.7290
Epoch 103/250
780/804 [============================>.] - ETA: 0s - loss: 0.5561 - accuracy: 0.7282
Epoch 103: saving model to checkpoints\Checkpoints.103.hdf5
804/804 [==============================] - 1s 787us/step - loss: 0.5555 - accuracy: 0.7285
Epoch 104/250


Epoch 133/250
782/804 [============================>.] - ETA: 0s - loss: 0.5594 - accuracy: 0.7225
Epoch 133: saving model to checkpoints\Checkpoints.133.hdf5
804/804 [==============================] - 1s 785us/step - loss: 0.5591 - accuracy: 0.7228
Epoch 134/250
788/804 [============================>.] - ETA: 0s - loss: 0.5602 - accuracy: 0.7280
Epoch 134: saving model to checkpoints\Checkpoints.134.hdf5
804/804 [==============================] - 1s 779us/step - loss: 0.5611 - accuracy: 0.7274
Epoch 135/250
784/804 [============================>.] - ETA: 0s - loss: 0.5626 - accuracy: 0.7290
Epoch 135: saving model to checkpoints\Checkpoints.135.hdf5
804/804 [==============================] - 1s 781us/step - loss: 0.5630 - accuracy: 0.7287
Epoch 136/250
738/804 [==========================>...] - ETA: 0s - loss: 0.5632 - accuracy: 0.7267
Epoch 136: saving model to checkpoints\Checkpoints.136.hdf5
804/804 [==============================] - 1s 761us/step - loss: 0.5622 - accuracy: 0.7279


Epoch 166/250
740/804 [==========================>...] - ETA: 0s - loss: 0.5662 - accuracy: 0.7222
Epoch 166: saving model to checkpoints\Checkpoints.166.hdf5
804/804 [==============================] - 1s 759us/step - loss: 0.5654 - accuracy: 0.7224
Epoch 167/250
737/804 [==========================>...] - ETA: 0s - loss: 0.5656 - accuracy: 0.7238
Epoch 167: saving model to checkpoints\Checkpoints.167.hdf5
804/804 [==============================] - 1s 761us/step - loss: 0.5641 - accuracy: 0.7257
Epoch 168/250
740/804 [==========================>...] - ETA: 0s - loss: 0.5623 - accuracy: 0.7283
Epoch 168: saving model to checkpoints\Checkpoints.168.hdf5
804/804 [==============================] - 1s 760us/step - loss: 0.5628 - accuracy: 0.7283
Epoch 169/250
740/804 [==========================>...] - ETA: 0s - loss: 0.5642 - accuracy: 0.7288
Epoch 169: saving model to checkpoints\Checkpoints.169.hdf5
804/804 [==============================] - 1s 759us/step - loss: 0.5640 - accuracy: 0.7287


Epoch 199/250
741/804 [==========================>...] - ETA: 0s - loss: 0.5635 - accuracy: 0.7276
Epoch 199: saving model to checkpoints\Checkpoints.199.hdf5
804/804 [==============================] - 1s 757us/step - loss: 0.5631 - accuracy: 0.7282
Epoch 200/250
741/804 [==========================>...] - ETA: 0s - loss: 0.5652 - accuracy: 0.7255
Epoch 200: saving model to checkpoints\Checkpoints.200.hdf5
804/804 [==============================] - 1s 759us/step - loss: 0.5648 - accuracy: 0.7259
Epoch 201/250
790/804 [============================>.] - ETA: 0s - loss: 0.5639 - accuracy: 0.7247
Epoch 201: saving model to checkpoints\Checkpoints.201.hdf5
804/804 [==============================] - 1s 776us/step - loss: 0.5638 - accuracy: 0.7250
Epoch 202/250
741/804 [==========================>...] - ETA: 0s - loss: 0.5615 - accuracy: 0.7270
Epoch 202: saving model to checkpoints\Checkpoints.202.hdf5
804/804 [==============================] - 1s 761us/step - loss: 0.5617 - accuracy: 0.7269


Epoch 232/250
739/804 [==========================>...] - ETA: 0s - loss: 0.5639 - accuracy: 0.7275
Epoch 232: saving model to checkpoints\Checkpoints.232.hdf5
804/804 [==============================] - 1s 761us/step - loss: 0.5649 - accuracy: 0.7273
Epoch 233/250
742/804 [==========================>...] - ETA: 0s - loss: 0.5666 - accuracy: 0.7285
Epoch 233: saving model to checkpoints\Checkpoints.233.hdf5
804/804 [==============================] - 1s 759us/step - loss: 0.5662 - accuracy: 0.7287
Epoch 234/250
783/804 [============================>.] - ETA: 0s - loss: 0.5652 - accuracy: 0.7282
Epoch 234: saving model to checkpoints\Checkpoints.234.hdf5
804/804 [==============================] - 1s 785us/step - loss: 0.5650 - accuracy: 0.7285
Epoch 235/250
778/804 [============================>.] - ETA: 0s - loss: 0.5649 - accuracy: 0.7248
Epoch 235: saving model to checkpoints\Checkpoints.235.hdf5
804/804 [==============================] - 1s 786us/step - loss: 0.5652 - accuracy: 0.7247


In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5680 - accuracy: 0.7270 - 207ms/epoch - 773us/step
Loss: 0.5679751038551331, Accuracy: 0.7269970774650574


In [17]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization4.h5')